In [31]:
import os
import pandas as pd
import numpy as np
from scipy.stats import norm, chi2, percentileofscore
import json
import sys
import math
import matplotlib.pyplot as plt

# Load the DB file
# df_db : Data frame of accumulated Experimental result information - Abundance
path_db = os.path.abspath('') + "/input/db_abundance.csv"
df_db = pd.read_csv(path_db)

# Load the Experiment result file
# df_exp : Data frame of Experimental result information - Abundance
path_exp = os.path.abspath('') + "/input/experiment_result_abundance.csv"
df_exp = pd.read_csv(path_exp)

# Load the merged Valencia output file
# df_valencia : Data frame of merged Valencia output
path_valencia = os.path.abspath('') + "/input/VALENCIA_output_merged.csv"
df_valencia = pd.read_csv(path_valencia)

# Insert data into DB - Merge the data frame df_db & df_exp

try: 
    df_db = pd.merge(df_db, df_exp, how='outer',on='taxa', suffixes=['', '_right']) 
    df_db = df_db.fillna(0)
    df_db = df_db.filter(regex='^(?!.*_right).*') # Eliminate duplicate columns

    df_db_rev = df_db.set_index(keys=['taxa'], inplace=False, drop=True)    
    df_db_rev.to_csv(path_db)
    
except:
    print("Check the Experiment result file")
    sys.exit()

    
# Delete the diversity, observed rows
if (list(df_exp['taxa'][0:2]) == ['diversity', 'observed']) & (list(df_db['taxa'][0:2]) == ['diversity', 'observed']):
    df_exp = df_exp.iloc[2:,:]
    df_db = df_db.iloc[2:,:]
else:
    print("Check the diversity & observed rows in the exp file or db file")
    sys.exit()


# Load the Phenotype-Microbiome file
# df_beta : Data frame of of Phenotype-Microbiome information
path_beta = os.path.abspath('') + "/input/phenotype_microbiome.xlsx"
df_beta = pd.read_excel(path_beta)
df_beta.rename(columns = {"Disease": "phenotype", "NCBI name": "ncbi_name", "MIrROR name": "microbiome", "Health sign": "beta", "subtract": "microbiome_subtract"}, inplace=True)
df_beta = df_beta[["phenotype", "ncbi_name", "microbiome", "beta","microbiome_subtract"]]
df_beta['beta'] = df_beta['beta'].replace({'증가': 1, '감소': -1})

li_new_sample_name = list(df_exp.columns)[1:]  
li_phenotype = list(dict.fromkeys(df_beta['phenotype']))

## Top 5 NCBI name 
li_phenotype_ncbi_name = []

for idx, row in df_beta.iterrows(): 
    if [row['phenotype'], row['ncbi_name']] not in li_phenotype_ncbi_name:
        li_phenotype_ncbi_name.append([row['phenotype'], row['ncbi_name']])

json_abundance = []

for i in range(len(li_new_sample_name)):
    for j in range(len(li_phenotype_ncbi_name)):
        
        condition_phen = (df_beta.phenotype == li_phenotype_ncbi_name[j][0]) & (df_beta.ncbi_name == li_phenotype_ncbi_name[j][1]) & (df_beta.beta == 1) 

        abundance = 0 
        for idx_beta, row_beta in df_beta[condition_phen].iterrows(): 
            if row_beta['microbiome'][:3] in ['s__', 'g__']:
                condition = (df_exp.taxa == row_beta['microbiome'])
                if len(df_exp[condition]) > 0:
                    abundance += df_exp[condition][li_new_sample_name[i]].values[0]

                    if (pd.isna(row_beta['microbiome_subtract']) is False):
                        li_micro_sub = row_beta['microbiome_subtract'].split('\n')
                        for micro_sub in li_micro_sub:
                            condition_sub = (df_exp.taxa == micro_sub)
                            if len(df_exp[condition_sub]) > 0:
                                 abundance -= df_exp[condition_sub][li_new_sample_name[i]].values[0]
                            
                json_abundance.append({"sample_name" : li_new_sample_name[i], "phenotype" : li_phenotype_ncbi_name[j][0], "ncbi_name" : li_phenotype_ncbi_name[j][1], "abundance" : abundance})
                
df_abundance = pd.DataFrame.from_dict(json_abundance)   

df_abundance = df_abundance.drop_duplicates(['sample_name', 'phenotype', 'ncbi_name'], keep='last')

df_top_five = pd.DataFrame(columns = ["sample_name", "phenotype", "ncbi_name","abundance"])

for i in range(len(li_new_sample_name)):
    for j in range(len(li_phenotype)):
    
        condition = (df_abundance.sample_name == li_new_sample_name[i]) & (df_abundance.phenotype == li_phenotype[j])
        df_new = df_abundance[condition].sort_values(by=['abundance'], ascending=False).head(5)
        df_top_five = pd.concat([df_top_five,df_new])

df_top_five = df_top_five.set_index(keys=['sample_name'], inplace=False, drop=True)           
df_top_five.to_excel('/home/kbkim/vaginal_microbiome/output/top5.xlsx')


# Subtract the abundance - df_exp

for idx_beta, row_beta in df_beta.iterrows(): 
    li_micro_sub = []

    if pd.isna(row_beta['microbiome_subtract']) is False:
        li_micro_sub = row_beta['microbiome_subtract'].split('\n')
        
        for micro_sub in li_micro_sub:
            condition = (df_exp.taxa == row_beta['microbiome'])
            condition_sub = (df_exp.taxa == micro_sub)
            
            if len(df_exp[condition_sub]) > 0:
                
                for sample_name in li_new_sample_name:
                    df_exp.loc[condition, sample_name] -= df_exp[condition_sub][sample_name].values[0]

# Calculate the GRS 
# li_phenotype : Phenotype list 
# df_grs : Data frame of grs corresponding to specific phenotype and sample

df_grs = pd.DataFrame(index = li_new_sample_name, columns = li_phenotype)
df_grs = df_grs.fillna(0) 

for i in range(len(li_new_sample_name)):
    for j in range(len(li_phenotype)):
        condition_phen = (df_beta.phenotype == li_phenotype[j])   
        grs = 0
        
        for idx_beta, row_beta in df_beta[condition_phen].iterrows():
            condition_micro = (df_exp.taxa == row_beta['microbiome'])
            
            if (len(df_exp[condition_micro]) > 0):      
                x_i = df_exp[condition_micro][li_new_sample_name[i]].values[0]
                ln_x_i = math.log(x_i + 1e-15)  
                grs += ln_x_i * row_beta['beta']
            
            elif (len(df_exp[condition_micro]) == 0):      
                x_i = 0
                ln_x_i = math.log(x_i + 1e-15)  
                grs += ln_x_i * row_beta['beta']                
            
        grs /= len(df_beta[condition_phen])       
        df_grs.loc[li_new_sample_name[i], li_phenotype[j]] = grs

# Load the grs_db dataframe
# df_grs_db : Data frame of grs in the DB
path_grs_db = os.path.abspath('') + "/input/vaginal_grs.xlsx"
df_grs_db = pd.read_excel(path_grs_db, index_col=0)        

# Calculation - Percentile Rank
df_percentile_rank = pd.DataFrame(index = li_new_sample_name, columns = li_phenotype)

for i in range(len(li_new_sample_name)):
    for j in range(len(li_phenotype)):
        df_percentile_rank.loc[li_new_sample_name[i], li_phenotype[j]] = (percentileofscore(list(df_grs_db[li_phenotype[j]]), df_grs.loc[li_new_sample_name[i], li_phenotype[j] ], kind='mean')).round(1)

# Outliers
for i in range(len(li_phenotype)):
    df_percentile_rank.loc[df_percentile_rank[li_phenotype[i]]<=5, li_phenotype[i]] = 5.0
    df_percentile_rank.loc[df_percentile_rank[li_phenotype[i]]>=95, li_phenotype[i]] = 95.0

df_percentile_rank = df_percentile_rank.fillna('None')
        
# Merge the valencia output file & df_probabilities

df_percentile_rank = pd.merge(df_percentile_rank, df_valencia[['sampleID', 'subCST', 'score', 'CST']], how='left',left_index=True, right_on = 'sampleID') 
df_percentile_rank = df_percentile_rank.set_index(keys=['sampleID'], inplace=False, drop=True)    

# Save the output file - Probabilities calculated by estimating population variance samples

path_vaginal_probability_sample_estimation_output = os.path.abspath('') + "/output/vaginal_probability_sample_estimation.xlsx"
df_percentile_rank.to_excel(path_vaginal_probability_sample_estimation_output)

print('Analysis Complete')       
        

Analysis Complete


In [32]:
df_percentile_rank

,Pelvic Inflammatory Diseases,Endometritis,Miscarriage,Ovarian Cancer,Cervical Intraepithelial Neoplasia,Preterm Prelabor Rupture of Membranes (PPROM),Dysmenorrhea (Menstrual pain),Adenomyosis,Vaginal Dryness,Gestational Diabetes,subCST,score,CST
sampleID,,,,,,,,,,,,,
20230116_BC05,5.0,37.5,89.3,92.0,58.0,29.5,54.5,75.0,44.6,83.9,III-B,0.072628,III
20230116_BC02,5.0,9.8,54.5,92.0,32.1,5.0,29.5,46.4,5.4,50.0,IV-B,0.418956,IV-B
20230116_BC03,5.0,57.1,66.1,70.5,50.9,80.4,44.6,65.2,75.9,5.0,III-B,0.779614,III
20230116_BC12,5.0,64.3,5.0,27.7,5.0,39.3,5.0,6.2,28.6,90.2,IV-C1,0.419117,IV-C
20230116_BC21,5.0,5.0,19.6,58.0,54.5,10.7,8.9,16.1,70.5,21.4,V,0.881183,V
20230116_BC07,5.0,12.5,21.4,92.0,46.4,7.1,17.0,70.5,25.0,75.0,IV-C0,0.012731,IV-C
20230116_BC11,5.0,5.4,52.7,25.9,67.0,63.4,78.6,6.2,72.3,5.0,IV-C0,0.094249,IV-C
20230116_BC01,5.0,20.5,67.9,17.0,91.1,53.6,81.2,6.2,19.6,58.0,IV-A,0.040625,IV-A
20230116_BC04,5.0,74.1,78.6,81.2,95.0,88.4,92.0,6.2,45.5,62.5,IV-B,0.618101,IV-B


In [33]:
df_grs_db

,Pelvic Inflammatory Diseases,Endometritis,Miscarriage,Ovarian Cancer,Cervical Intraepithelial Neoplasia,Preterm Prelabor Rupture of Membranes (PPROM),Dysmenorrhea (Menstrual pain),Adenomyosis,Vaginal Dryness,Gestational Diabetes
20230116_BC05,-34.538776,-24.535127,-19.007868,0.000000,-6.813467,-22.837514,-16.478956,-28.971574,-11.382317,-12.971992
20230116_BC02,-34.538776,-25.511043,-25.906646,0.000000,-13.817561,-30.708326,-18.817418,-29.546276,-18.596881,-15.141610
20230116_BC03,-34.538776,-24.026066,-21.529225,-4.110846,-10.270767,-15.094988,-17.726801,-29.140365,-8.031523,-17.293292
20230116_BC12,-34.538776,-23.920989,-34.538776,-5.304035,-25.710802,-21.625525,-21.916208,-34.538776,-13.872296,-12.936797
20230116_BC21,-34.538776,-26.112207,-28.467116,-4.316013,-9.110455,-23.801275,-20.640097,-32.110112,-9.129453,-16.131876
20230116_BC07,-34.538776,-25.316118,-28.381822,0.000000,-10.733510,-26.758045,-19.366613,-29.124933,-13.907652,-13.927610
20230116_BC11,-34.538776,-25.591302,-26.908524,-5.328096,-4.630893,-16.832237,-13.859194,-34.538776,-8.991965,-19.251598
20230116_BC01,-34.538776,-24.871322,-21.219019,-8.810681,1.241988,-18.272095,-13.838798,-34.538776,-14.667410,-14.285151
20230116_BC04,-34.538776,-23.817224,-19.834583,-0.667889,3.464132,-13.587188,-13.509810,-34.538776,-11.295602,-14.249139
20230116_BC06,-34.538776,-25.564766,-28.328659,0.000000,-15.755172,-22.778143,-19.300545,-29.138934,-6.511699,-14.959456


In [10]:
def percentile_rank_interpolation(data, value):
    """
    Calculates the percentile rank for a value in non-normal distribution data using the interpolation method.
    If the value is not in the data, the percentile rank of the nearest value is returned.

    Parameters:
    -----------
    data : array_like
        Input data. Must be 1-dimensional.
    value : float
        Value for which to calculate the percentile rank.

    Returns:
    --------
    result : float
        Percentile rank of the given value in the dataset.
    """
    # Sort the data in ascending order
    sorted_data = np.sort(data)

    # Find the index of the value if it is in the data
    try:
        index = np.where(sorted_data == value)[0][0]
        rank = 1 + index
        percentile = 100 * (rank - 0.5) / len(sorted_data)
        
        result = round(percentile, 1)
        
        if result >= 95:
            result = 95.0
        elif result <= 5:
            result = 5.0    
        
        return result
        
    except IndexError:
        # If the value is not in the data, find the index of the nearest value
        index = (np.abs(sorted_data - value)).argmin()

        # Calculate the rank of the value
        rank = 1 + index

        # Calculate the percentile corresponding to the rank
        percentile = 100 * (rank - 0.5) / len(sorted_data)
        print('percentile', percentile)


        # Interpolate between adjacent percentile ranks
        lower_percentile = int(percentile)
        upper_percentile = lower_percentile + 1
        print('lower_percentile', lower_percentile)
        print('upper_percentile', upper_percentile)
        
        lower_rank = int(np.floor((lower_percentile / 100) * len(sorted_data)))
        upper_rank = int(np.ceil((upper_percentile / 100) * len(sorted_data)))
        print('lower_rank', lower_rank)
        print('upper_rank', upper_rank)
        if upper_rank >= len(sorted_data):
            lower_value = sorted_data[lower_rank-1]
            upper_value = sorted_data[upper_rank-1]
        else:
            lower_value = sorted_data[lower_rank]
            upper_value = sorted_data[upper_rank]            

        print('lower_value', lower_value)
        print('upper_value', upper_value)
        
        if upper_value != lower_value:
            result = lower_percentile + (value - lower_value) / (upper_value - lower_value) * (upper_percentile - lower_percentile)
        else:
            result = lower_percentile

        # Outlier
        if result >= 95:
            result = 95.0
        elif result <= 5:
            result = 5.0   
        
        # Round    
        result = round(result, 1)
        print('result', result)    
            
    return result

In [11]:
data = [1.0, 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 5, 10, 200, 300]
value = 1.0
percentile_rank_interpolation(data, value)

5.0

In [12]:
data = [1.0, 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 5, 10, 200, 300]
value = 1.09
percentile_rank_interpolation(data, value)

percentile 65.38461538461539
lower_percentile 65
upper_percentile 66
lower_rank 8
upper_rank 9
lower_value 1.08
upper_value 5.0
result 65.0


65.0

In [13]:
data = [1.0, 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 5, 10, 200, 300]
value = 200
percentile_rank_interpolation(data, value)

88.5

In [14]:
data = [1.0, 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 5, 10, 200, 300]
value = 210
percentile_rank_interpolation(data, value)

percentile 88.46153846153847
lower_percentile 88
upper_percentile 89
lower_rank 11
upper_rank 12
lower_value 200.0
upper_value 300.0
result 88.1


88.1

In [298]:
data = [1.0, 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 5, 10]
value = 1.03
percentile_rank_interpolation(data, value)

31.8

In [287]:
data = [1.0, 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 5, 10]
value = 9
percentile_rank_interpolation(data, value)

percentile 95.45454545454545
lower_percentile 95
upper_percentile 96
lower_rank 10
upper_rank 11
lower_value 5.0
upper_value 10.0
result 95.0


95.0

In [288]:
data = [1, 2, 3, 3, 4]
value = 3
percentile_rank_interpolation(data, value)

50.0

In [289]:
data = [1, 2, 3, 4]
value = 3
percentile_rank_interpolation(data, value)

62.5

In [8]:
from scipy import stats
stats.percentileofscore([1.0, 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 5, 10], 1.03, kind='mean')

31.81818181818182